In [2]:
# Uninstall potentially conflicting packages
!pip uninstall -y transformers accelerate unsloth torch torchvision torchaudio

# Install base packages
!pip install unsloth

# Install dependencies
!pip install -q transformers accelerate peft
!pip install -q datasets evaluate bitsandbytes trl
!pip install -q torch torchvision torchaudio

# Install Colab-optimized unsloth
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install other tools
!pip install pandas scikit-learn
!pip install -q ipywidgets

Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: accelerate 1.1.1
Uninstalling accelerate-1.1.1:
  Successfully uninstalled accelerate-1.1.1
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 92.4 MB/s eta 0:00:00
Found existing installation: unsloth 2024.12.4
Uninstalling unsloth-2024.12.4:
  Successfully uninstalled unsloth-2024.12.4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-upu9twkj/unsloth_4818aed743354c95b2d9de1c7e50bd44
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-upu9twkj/unsloth_4818aed743354c95b2d9de1c7e50bd44
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.4-py3-none-any.whl size=173746 sha256=cb8cc23fbbe0c1ab12e287863b16fa

Preparations
------

In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Environment setup
import os
import warnings
import random
import numpy as np
import torch
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import gc
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments, TrainerCallback
import transformers
import accelerate
import json
import openpyxl
from datasets import Dataset
import gdown

# Print versions
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")

# Configure environment
os.environ["CUDA_VISIBLE_DEVICES"]="0"
warnings.filterwarnings('ignore')
torch.set_float32_matmul_precision('high')

# Set random seeds
def set_seeds(seed=3407):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# Memory management utilities
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()

def print_gpu_utilization():
    print("\nGPU Memory Usage:")
    !nvidia-smi | grep -E "Memory|Volatile"

def print_detailed_gpu_info():
    print("\nDetailed GPU Memory Info:")
    print(f"Allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
    print(f"Cached: {torch.cuda.memory_reserved()/1024**2:.2f} MB")
    print(f"Max Allocated: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")

Mounted at /content/drive
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
PyTorch version: 2.5.1+cu124
Transformers version: 4.47.1
Accelerate version: 1.2.1


set up wandb

In [ ]:
!pip install wandb --upgrade

import wandb
wandb.login()

define the sweep

In [ ]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'
    }

sweep_config['metric'] = metric

parameters_dict = {
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
        },
    'warmup_ratio': {
        'values': [0.05, 0.1, 0.2]
        },
    'weight_decay': {
        'values': [0.01, 0.03, 0.05]
        },
    'per_device_train_batch_size': {
        'values': [2, 4]
        },
    'gradient_accumulation_steps': {
        'values': [2, 4, 8]
        },
    'epochs': {
        'value': 1
        }
}

sweep_config['parameters'] = parameters_dict

In [ ]:
import pprint

pprint.pprint(sweep_config)

Define class
---

In [12]:
class MemoryCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 50 == 0:  # 每50步清理一次
            clear_memory()
            print_detailed_gpu_info()

class AIGenerationDetector:
    def __init__(self, max_seq_length=2048, save_dir='/content/drive/MyDrive/ai_detection_model', data_dir='/content/drive/MyDrive/ai_dataset'):
        """
        Initializes the AIGenerationDetector class.
        Args:
            max_seq_length (int): Maximum sequence length for the model.
            save_dir (str): Directory to save the model and checkpoints.
            data_dir (str): Directory to save the dataset.
        """
        self.max_seq_length = max_seq_length
        self.save_dir = save_dir
        self.data_dir = data_dir
        self.model = None
        self.tokenizer = None
        self.train_dataset = None
        self.eval_dataset = None
        self.test_dataset = None
        os.makedirs(self.save_dir, exist_ok=True)
        os.makedirs(self.data_dir, exist_ok=True)


    def download_data(self):
        """
        Downloads the dataset from Google Drive only if the files don't exist.
        """
        # Download target folder path
        download_dir = self.data_dir

        # human.xlsx and ai.xlsx Google Drive download links
        human_xlsx_url = "https://drive.google.com/uc?export=download&id=1EXEs6bDZ8KYWN6wei17QTOFKT8TOHVbq"
        ai_xlsx_url = "https://drive.google.com/uc?export=download&id=1fGU3P_xAfqHZeYQiwE382YmKiQZoWmFi"

        # File names
        human_xlsx_filename = "human.xlsx"
        ai_xlsx_filename = "ai.xlsx"

        # Check if human.xlsx exists
        human_xlsx_output_path = os.path.join(download_dir, human_xlsx_filename)
        if not os.path.exists(human_xlsx_output_path):
            print(f"Downloading {human_xlsx_filename} from Google Drive to {download_dir}...")
            gdown.download(human_xlsx_url, human_xlsx_output_path, quiet=False)
        else:
            print(f"{human_xlsx_filename} already exists in {download_dir}. Skipping download.")

        # Check if ai.xlsx exists
        ai_xlsx_output_path = os.path.join(download_dir, ai_xlsx_filename)
        if not os.path.exists(ai_xlsx_output_path):
            print(f"Downloading {ai_xlsx_filename} from Google Drive to {download_dir}...")
            gdown.download(ai_xlsx_url, ai_xlsx_output_path, quiet=False)
        else:
            print(f"{ai_xlsx_filename} already exists in {download_dir}. Skipping download.")

        print("Data download check complete!")

    def setup_model(self):
        """
        Loads the pre-trained model and tokenizer and configures the PEFT model.
        """
        clear_memory()
        print("Loading model...")

        try:
            model, tokenizer = FastLanguageModel.from_pretrained(
                model_name="unsloth/Meta-Llama-3.1-8B",
                max_seq_length=self.max_seq_length,
                load_in_4bit=True,
            )

            model = FastLanguageModel.get_peft_model(
                model,
                r=16,
                target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
                lora_alpha=16,
                lora_dropout=0.1,
                bias="none",
                use_gradient_checkpointing=True,
                random_state=3407,
                use_rslora=True,
            )

            self.model = model
            self.tokenizer = tokenizer
            print("Model loaded successfully!")

        except Exception as e:
            print(f"Error loading model: {str(e)}")
            raise

    def prepare_datasets(self, max_samples=1000):
        """
        Prepares the training, evaluation, and test datasets.
        Args:
            max_samples (int): Maximum number of training samples to use.
        """
        clear_memory()
        print("Preparing datasets...")
        try:
            human_xlsx_path = os.path.join(self.data_dir, "human.xlsx")
            ai_xlsx_path = os.path.join(self.data_dir, "ai.xlsx")

            if not os.path.exists(human_xlsx_path) or not os.path.exists(ai_xlsx_path):
                self.download_data()

            print(f"Loading data from {human_xlsx_path} and {ai_xlsx_path}...")

            df_human = pd.read_excel(human_xlsx_path)
            df_human = df_human[['abstract']]
            df_human['is_ai_generated'] = 'False'

            df_ai = pd.read_excel(ai_xlsx_path)
            df_ai = df_ai[['abstract']]
            df_ai['is_ai_generated'] = 'True'

            # merge
            df = pd.concat([df_human, df_ai], ignore_index=True)

            dataset = Dataset.from_pandas(df)
            print(f"Data loaded. Total samples: {len(dataset)}")

            # Take required number of samples, if specified
            if max_samples > 0:
                dataset = dataset.shuffle(seed=3407).select(range(min(max_samples, len(dataset))))

            # Split train, val and test set
            train_val_idx, test_idx = train_test_split(
                range(len(dataset)),
                test_size=0.2,
                random_state=3407
            )

            train_idx, val_idx = train_test_split(
                train_val_idx,
                test_size=0.125,
                random_state=3407
            )

            train_examples = [self.process_training_example(dataset[i]) for i in train_idx]
            eval_examples = [self.process_training_example(dataset[i]) for i in val_idx]
            test_examples = [dataset[i] for i in test_idx]

            self.train_dataset = Dataset.from_list(train_examples)
            self.eval_dataset = Dataset.from_list(eval_examples)
            self.test_dataset = Dataset.from_list([{'text': item['abstract']} for item in test_examples])

            del train_examples, eval_examples, dataset
            clear_memory()
            print(f"Datasets prepared! Train size: {len(self.train_dataset)}, Eval size: {len(self.eval_dataset)}, Test size: {len(self.test_dataset)}")

        except Exception as e:
            print(f"Error preparing datasets: {str(e)}")
            raise


    def process_training_example(self, example):
         """
         Processes a single training example to create the prompt.
         Args:
            example (dict): A dictionary containing the 'text' and 'is_ai_generated' fields.
         Returns:
            dict: A dictionary containing the processed text.
         """
         text = example['abstract']
         is_ai_generated = example['is_ai_generated']  # Get the field from the dataset

         prompt = (
            "You are an expert in distinguishing between text written by humans and text generated by AI.\n\n"
            f"Given Text: {text}\n\n"
            "Based on careful analysis, is the text generated by an AI? Respond with EXACTLY 'True' or 'False'.\n"
            f"Answer: {str(is_ai_generated)}"
         ) + self.tokenizer.eos_token

         return {"text": prompt}


    def process_test_example(self, example):
        """
        Processes a single test example to create the prompt.
        Args:
            example (dict): A dictionary containing the 'text' field.
        Returns:
            str: The generated prompt for testing.
        """
        #text = example['abstract']
        print(example)
        text=example

        prompt = (
           "You are an expert in distinguishing between text written by humans and text generated by AI.\n\n"
           f"Given Text: {text}\n\n"
           "Based on careful analysis, is the text generated by an AI? Respond with EXACTLY 'True' or 'False'.\n"
        )
        return prompt

    def setup_training_args(self, config=None):
        """
        Sets up training arguments, either default or for hyperparameter sweeping.
        Args:
           config (dict, optional): Configuration for hyperparameter sweep. Defaults to None.
        Returns:
           TrainingArguments: Training arguments based on the given configuration.
        """
        if config is None:
            # Default training arguments
            return TrainingArguments(
                output_dir=os.path.join(self.save_dir, "checkpoints"),
                per_device_train_batch_size=2,
                gradient_accumulation_steps=8,
                warmup_ratio=0.1,
                num_train_epochs=3,
                learning_rate=0.0006026,
                fp16=True,
                logging_steps=10,
                optim="adamw_torch",
                weight_decay=0.05,
                lr_scheduler_type="cosine",
                seed=3407,
                evaluation_strategy="steps",
                eval_steps=50,
                save_strategy="steps",
                save_steps=50,
                load_best_model_at_end=True,
                metric_for_best_model="eval_loss",
                gradient_checkpointing=True,
                max_grad_norm=0.3,
                report_to="none",
                remove_unused_columns=True,
                dataloader_pin_memory=False
            )
        else:
            # Training arguments for hyperparameter sweep
            return TrainingArguments(
                output_dir=os.path.join(self.save_dir, "checkpoints"),
                per_device_train_batch_size=config.per_device_train_batch_size,
                gradient_accumulation_steps=config.gradient_accumulation_steps,
                warmup_ratio=config.warmup_ratio,
                num_train_epochs=config.epochs,
                learning_rate=config.learning_rate,
                fp16=True,
                logging_steps=10,
                optim="adamw_torch",
                weight_decay=config.weight_decay,
                lr_scheduler_type="cosine",
                seed=3407,
                evaluation_strategy="steps",
                eval_steps=50,
                save_strategy="steps",
                save_steps=50,
                load_best_model_at_end=True,
                metric_for_best_model="eval_loss",
                gradient_checkpointing=True,
                max_grad_norm=0.3,
                report_to="wandb",
                remove_unused_columns=True,
                dataloader_pin_memory=False,
            )

    def train(self):
        """
        Trains the model using the SFTTrainer.
        """
        clear_memory()
        print("Starting training...")

        try:
            trainer = SFTTrainer(
                model=self.model,
                tokenizer=self.tokenizer,
                train_dataset=self.train_dataset,
                eval_dataset=self.eval_dataset,
                dataset_text_field="text",
                max_seq_length=self.max_seq_length,
                dataset_num_proc=2,
                packing=False,
                args=self.setup_training_args(),
                callbacks=[MemoryCallback()]
            )

            trainer.train()

            final_save_path = os.path.join(self.save_dir, "final_model")
            self.model.save_pretrained(final_save_path)
            self.tokenizer.save_pretrained(final_save_path)
            print(f"Training completed! Model saved to {final_save_path}")

        except Exception as e:
            print(f"Error during training: {str(e)}")
            raise


    def sweep(self):
       """
        Conducts a hyperparameter sweep using Weights & Biases.
        """
       print("Starting sweeping...")

       with wandb.init():
            config = wandb.config
            training_args = self.setup_training_args(config)

            try:
                trainer = SFTTrainer(
                    model=self.model,
                    tokenizer=self.tokenizer,
                    train_dataset=self.train_dataset,
                    eval_dataset=self.eval_dataset,
                    dataset_text_field="text",
                    max_seq_length=self.max_seq_length,
                    dataset_num_proc=2,
                    packing=False,
                    args=training_args
                    )
                trainer.train()

                final_save_path = os.path.join(self.save_dir, "final_model")
                self.model.save_pretrained(final_save_path)
                self.tokenizer.save_pretrained(final_save_path)
                print(f"Training completed! Model saved to {final_save_path}")

            except Exception as e:
                  print(f"Error during training: {str(e)}")
                  raise


    def generate_predictions(self, batch_size=16):
         """
        Generates predictions on the test dataset.
        Args:
           batch_size (int): Batch size for generating predictions.
        Returns:
           list: List of prediction values.
        """
         clear_memory()
         print("Generating predictions...")

         try:
            FastLanguageModel.for_inference(self.model)
            predictions = []
            # Convert test data to a list to support batch processing
            test_examples = list(self.test_dataset)
            total_batches = (len(test_examples) + batch_size - 1) // batch_size
            all_predictions = []

            # Process in batches
            for i in range(0, len(test_examples), batch_size):
                if i % (batch_size * 10) == 0:
                    print(f"Processing batch {i//batch_size}/{total_batches}")

                # Get current batch samples
                batch = test_examples[i:i + batch_size]
                prompts = [self.process_test_example(example) for example in batch]

                # Batch encoding
                inputs = self.tokenizer(
                    prompts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=self.max_seq_length
                ).to("cuda")

                # Batch generation
                with torch.inference_mode():
                    outputs = self.model.generate(
                        **inputs,
                        max_new_tokens=8,  # Reduce the number of generated tokens as we only need True/False
                        temperature=0.1,
                        top_p=0.9,
                        do_sample=False,    # Disable sampling for faster generation
                        use_cache=True,
                        pad_token_id=self.tokenizer.pad_token_id,
                    )

                input_length = inputs['input_ids'].shape[1]
                responses = self.tokenizer.batch_decode(
                    [output[input_length:] for output in outputs],
                    skip_special_tokens=True
                )

                # Batch processing prediction results
                batch_predictions = ["true" in response.lower() for response in responses]
                all_predictions.extend(batch_predictions)

                # Periodically clear memory
                if i % (batch_size * 50) == 0:
                    clear_memory()

            print(f"Total predictions: {len(all_predictions)}")
            assert len(all_predictions) == len(test_examples)

            return all_predictions

         except Exception as e:
            print(f"Error generating predictions: {str(e)}")
            raise


    def create_submission(self):
        """
        Creates the submission file in CSV format using the generated predictions.
        """
        print("Creating submission file...")
        try:
            predictions = self.generate_predictions(batch_size=16)
            print(f"Generated predictions: {len(predictions)}")

            assert len(predictions) == len(self.test_dataset), \
                f"Prediction count mismatch! Expected {len(self.test_dataset)}, got {len(predictions)}"

            submission_df = pd.DataFrame({
                'ID': range(len(predictions)),
                'is_ai_generated': predictions
            })

            print(f"Submission DataFrame shape: {submission_df.shape}")

            submission_path = os.path.join(self.save_dir, 'submission.csv')
            submission_df.to_csv(submission_path, index=False)
            print(f"Submission saved to {submission_path}")

            saved_df = pd.read_csv(submission_path)
            print(f"Saved file shape: {saved_df.shape}")

        except Exception as e:
            print(f"Error creating submission: {str(e)}")
            raise

# Hyper Parameters sweeping

## 1. Initial the sweep

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="ai_detection")

## 2. Run sweep agent

In [ ]:
def run_sweep(data_dir):
    """
    Runs the hyperparameter sweep.
    Args:
      data_dir(str): Path to the data directory.
    """
    trainer = AIGenerationDetector(data_dir = data_dir) # set data directory path
    trainer.download_data()
    trainer.setup_model()
    trainer.prepare_datasets(max_samples=1000)
    trainer.sweep()

In [ ]:
wandb.agent(sweep_id, run_sweep, count = 50)

# Get final result

Before running the main function, hyper parameter in trainer.train() should be changed.

In [13]:
def main(data_dir):
    """
    Main training and evaluation pipeline.
    Args:
       data_dir (str): Path to data directory.
    """
    try:
        set_seeds()
        print("Starting training pipeline...")

        # Initialize and run trainer
        trainer = AIGenerationDetector(data_dir=data_dir)  # set data directory path
        trainer.download_data()
        trainer.setup_model()
        trainer.prepare_datasets(max_samples=5000)
        trainer.train()
        trainer.create_submission()

        print("Training pipeline completed successfully!")

    except Exception as e:
        print(f"Fatal error in main: {str(e)}")
        raise
    finally:
        clear_memory()
        print_gpu_utilization()
        print_detailed_gpu_info()

# Main execution block
if __name__ == "__main__":
    data_dir = "/content/drive/MyDrive/ai_dataset"  # Set the Google Drive path to save data

    # Run the final training using main()
    main(data_dir)

Starting training pipeline...
human.xlsx already exists in /content/drive/MyDrive/ai_dataset. Skipping download.
ai.xlsx already exists in /content/drive/MyDrive/ai_dataset. Skipping download.
Data download check complete!
Loading model...
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model loaded successfully!
Preparing datasets...
Loading data from /content/drive/MyDrive/ai_dataset/human.xlsx and /content/drive/MyDrive/ai_dataset/ai.xlsx...
Data loaded. Total samples: 50670
Datasets prepared! Train size: 3500, Eval size: 500, Test size: 1000
Starting training...


Map (num_proc=2):   0%|          | 0/3500 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,500 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 654
 "-____-"     Number of trainable parameters = 13,631,488


Step,Training Loss,Validation Loss
50,1.739000,1.728870
100,1.767000,1.735386
150,1.772300,1.750454
200,1.738000,1.742761
250,1.568300,1.756627
300,1.585400,1.757929
350,1.638200,1.746924
400,1.616800,1.739211
450,1.352900,1.787875
500,1.339300,1.811091



Detailed GPU Memory Info:
Allocated: 6151.94 MB
Cached: 6172.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.94 MB
Cached: 6174.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.93 MB
Cached: 6172.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.93 MB
Cached: 6172.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.94 MB
Cached: 6172.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.94 MB
Cached: 6174.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.94 MB
Cached: 6172.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.94 MB
Cached: 6172.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.95 MB
Cached: 6174.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.94 MB
Cached: 6172.00 MB
Max Allocated: 7534.30 MB

Detailed GPU Memory Info:
Allocated: 6151.93 MB
Cached: 617